# Lecture 3

## Python Libraries (Review)

One of the great things about python is the incredible libraries that are either already installed with python or easily to install. If you want to do something in python, you can probably find a library or a GitHub page that already does it. 

Two resources of libraries are:

* [The Python Standard Library reference](https://docs.python.org/3.9/library/index.html): These packages are most likely already available in your python installation.
* [PyPI Repository](https://pypi.org): These packages can be installed easily using the `pip install` command.

### `import`

Jupyter notebooks execute code when a code cell is "run". You can run all of the cells sequentially, or manually run specific cells. All of our variable assignments and function definitions are executed in the python interperter that jupyter is running in that session. In a real piece of software, the code would likely be organized into files. 

For example, consider the checkers example we are working with. We can copy all of the relevant code from that lecture into one file: [checkers.py](./checkers.py). 

Now we can run the checkers game by "importing" the checkers "module":

In [ ]:
import checkers

We now have an "checkers" object in our current context:

In [ ]:
print(checkers)
print(type(checkers))

We can see the contents of the module using the `dir` built-in:

In [ ]:
dir(checkers)

Note that calling `dir()` without an argument will show you everything in your current context:

In [ ]:
x=1
dir()

To call something from the checkers module, we simply do, for example:

In [ ]:
checkers.make_game_board()

This may be combersome, so we can import a specific part of the checkers into our current context:

In [ ]:
from checkers import make_game_board
print(make_game_board())
dir()

We can rename what we import in our current context:

In [ ]:
from checkers import moves as checkers_moves
print(checkers_moves)

Or we can import everything into our current context using `*`:

In [ ]:
from checkers import *
dir()

## Reloading Modules

Note that you can change contents of the libraries that you load:

In [ ]:
checkers.size=10

In [ ]:
print(checkers.size)

In [ ]:
checkers.get_size()

In general this isn't good practice. But it does illustrate an important python behavior. 

`import` does not load a library again if it has already been loaded:

In [ ]:
import checkers

In [ ]:
print(checkers.size)

In [ ]:
checkers.get_size()

However, you can explicitly reload libraries if needed:

In [ ]:
import importlib
importlib.reload(checkers)
checkers.size

In [ ]:
checkers.size

In [ ]:
checkers.get_size()

## Recursion

At the end of today's lecture we'll use a tree search algorithm to implement a computer checks player. We'll use recursion to tackle that problem, so lets take a bit of time to go over recursion. 

A recursive function is one that calls itself. There are certain algorithms or computations that are simpler to express recursively. For example factorial:

$$
n! = n \cdot (n-1)\cdot(n-2)\cdot ...\cdot 2 \cdot  1
$$

We express factorial recursively by realizing that $n! =  n \cdot (n-1)!$ and $1! = 1$.

Lets code it up:

In [ ]:
def factorial(n):
    if n == 1:
        return 1
    else:
        return n * factorial(n-1)

In [ ]:
factorial(50)

We can see how python is peforming the computation by not performing the multiplication simply keeping what is multiplied in a tuple:

In [ ]:
def factorial_0(n):
    if n == 1:
        return 1
    else:
        return (n , factorial_0(n-1))

In [ ]:
factorial_0(5)

Note that you don't need to use recursion to compute factorial:

In [ ]:
def factorial_1(n):
    prod = 1
    for e in range(1,n+1):
        prod*=e  
    return prod

factorial_1(50)

But sometimes it's much simpler to think through the implemenation recursively.

### Loops in recursion
There are various ways to implement loops recursively, for example:

In [ ]:
def countdown(n):
     print(n)
     if n == 0:
         return             # Terminate recursion
     else:
         countdown(n - 1)   # Recursive call
countdown(10)

Here is an implementation of python's `range` function using recursion:

In [ ]:
def rec_range(start,stop=None,step=1):
    # Properly handle default arguments
    if stop:
       pass
    else:
        stop=start
        start=0
        
    if start < stop:
        return [start] + rec_range(start+step,stop,step)
    else:
        return []

In [ ]:
rec_range(2,20,2)

### More recursive examples
Another algorithm that is often implemented recursively is the Fibonacci sequence $F_0=0$, $F_1=1$, $F_n = F_{n-1} + F_{n-2}$. Here an implementation:

In [ ]:
def recur_fibo(n):
   if n <= 1:
       return n
   else:
       return(recur_fibo(n-1) + recur_fibo(n-2))

Checking for a palindrome:

In [ ]:
def is_palindrome(word):
     if len(word) <= 1:
         return True
     else:
         return word[0] == word[-1] and is_palindrome(word[1:-1])

### Recursion Limit
You should be aware that python had a limit of how many times a function can call itself:

In [ ]:
from sys import getrecursionlimit
getrecursionlimit()
1000

Which you can increase if you need:

In [ ]:
from sys import setrecursionlimit
setrecursionlimit(2000)
getrecursionlimit()
2000

### Traversing a Tree
The reason we are looking at recursion now, is that is the easiest way to traverse tree-like data structures. Consider the following data structure:

In [ ]:
data = [1 ,[21,22], [23,[31,32]], [[33,34],[35,36]] ]
data

Imagine you needed to count how many numbers are stored in the structure (aka count the leaves in the tree). How would you do it?

Here an approach:
* Create a `leaf_count` function that takes a list as argument.
* In the function, keep a counter, initialized to 0.
* Iterate over the list. 
    * If you see a number add one to counter.
    * If you see a list, call `leaf_count` with that list, add results to counter.
* Return the counter value.


In [ ]:
def leaf_count(ll):
    count = 0
    for item in ll:
        if isinstance(item, list):
            count += leaf_count(item)
        else:
            count += 1

    return count

In [ ]:
leaf_count(data)

Performing the same operation non-recursively is much more complicated:

In [ ]:
def leaf_count_non_recursive(ll):
    count = 0
    stack = []
    current_ll = ll
    i = 0

    while True:
        if i == len(current_ll):
            if current_ll == ll:
                return count
            else:
                current_ll, i = stack.pop()
                i += 1
                continue

        if isinstance(current_ll[i], list):
            stack.append([current_ll, i])
            current_ll = current_ll[i]
            i = 0
        else:
            count += 1
            i += 1

In [ ]:
leaf_count_non_recursive(data)

## Checkers "AI" Player

Lets now attempt to build a "AI" checkers player to play against a human players. There are various ways to construct such a player. 

One method is to write an algorithm that given a board, considers all possible moves of a player, and selects the best move. However just considering one single move at a time would result in a very dumb algorithm, which for example would fail to detect when a move results in subsequently losing a piece. Therefore, instead of a single move, the algorithm should consider several moves ahead. For every possible move, it should consider every possible response from the other player, and then every possible subsequent move, and so on. All of these possibilities can be organized in a tree. As you can imagine, the number of possbilities grows exponentially as the algorithm attempts to think many moves ahead ... and so does the computational cost. 

Another method is to use a Machine Learning algorithm that learns from either observing played games or plays against itself or humans. Indeed a technique known as reinforcement learning has been used to create AI players that can beat the best human players in games, such as go, where brute-force tree search algorithms are too computationally prohibitive.

For our example here, we will use the first method.

First we need to create a means of rating a move. Everytime we consider a move, we score the resulting board and choose the move that gives the best score. An obvious, but not necessarily optimal, choice is to count pieces:


In [ ]:
# A scoring function
def score_board(board,player):
    return count_pieces(board,player)-count_pieces(board,switch_player(player))

Next, lets sketch out our algorithm. We need a function that starting from the current board, generates and rates all possible moves up to a depth. Lets think through it. Our algorithm will be recursive, meaning that it will be a function that calls itself. 

* Function should take as arguments
    * the board
    * the player who is trying to win
    * the player whose turn we are currently considering
    * and a max depth.
* Function returns a list of moves and scores.
    
* Algorithm logic: 
    * If the current depth is 0, we have gone far enough and we are done moving. Score the board.
    * Find all pieces for the current player
        * For each pieces consider all possible moves (right or left)
            * Check if the move is possible, if so
                * Keep track of the move (player, location, and direction)
                * Consider subseqent moves by recursively calling self.
                * Return the chain of moves and scores.
                
    
    

In [ ]:
import copy

def generate_moves(board,player,current_player,depth, keep_all_moves=False):
    if depth==0:
        return list(),score_board(board,player)

    moves=list()
    scores=list()
    
    # Look through the board for pieces belonging to a specific player
    for i in range(size):
        for j in range(size):
            if board[i][j]==current_player:
                
                # For each piece, consider moving left or right
                for move in [left_move,right_move]:
                    
                    # Create a new copy of the board
                    new_board=copy.deepcopy(board)
                    
                    # Check if its a valid move
                    if move_piece(new_board,current_player,(i,j),move,verbose=False):
                        # If so, generate subsequent moves
                        next_moves,score=generate_moves(new_board,player,
                                                        switch_player(current_player),
                                                        depth-1,
                                                        keep_all_moves=keep_all_moves)
                        
                        # Keep track of this move
                        this_move=[(current_player,(i,j),move)]
                        if keep_all_moves:
                            # For our purposes, we usually only need to keep the top move!
                            this_move.extend(next_moves)
                        moves.append(this_move)
                        scores.append(score)

    return moves,scores
            


### Quick side on use of `deepcopy`

Note that we used `copy.deepcopy` here to make copies of the board. The reason for this that python lists just keep references to their data. If we copy a list, we copy the references. Deep copy results in copies of all references also. Consider the following examples.

In [ ]:
# Assignment doesn't work
x = [[1,2],[2,3]]
y = x
y[0][1]=-1
x

In [ ]:
# Copy works, but ...
x = [1,2,3]
y = copy.copy(x)
y[0]=-1
x

In [ ]:
# Copy doesn't work for nested lists
x = [[1,2],[2,3]]
y = copy.copy(x)
y[0][1]=-1
x

In [ ]:
# Why use deep copy?
x = [[1,2],[2,3]]
y = copy.deepcopy(x)
y[0][1]=-1
x

### Test `generate_moves`

Lets see what generate moves does. Lets make a board:

In [ ]:
from checkers import *
board=make_game_board()
draw_board(board)

Now create all possible first moves for player 1, scoring to depth of 1. Well keep all subsequent moves so we can see them.

In [ ]:
possible_moves,scores=generate_moves(board,player_1,player_1,1,keep_all_moves=True)

Here are the possible moves:

In [ ]:
possible_moves

We see seven possible first moves. Lets increase depth to see what happens:

In [ ]:
possible_moves,scores=generate_moves(board,player_1,player_1,2,keep_all_moves=True)

In [ ]:
possible_moves

We see that for each of the seven first moves, there are seven responses by the other player.

Now lets look at the associated scores (remember depth is 2):

In [ ]:
scores

For each of the seven first moves and seven responses, no pieces are taken, so score is 0 in all scenarios.

Lets increase depth to 3 and see what happens:

In [ ]:
possible_moves,scores=generate_moves(board,player_1,player_1,3,keep_all_moves=True)
scores

Now we see that there are several first moves that could result in a positive score, if the other player moves a bad move.

Lets look at depth 5:

In [ ]:
possible_moves,scores=generate_moves(board,player_1,player_1,5)
scores

### Selecting a move

We now need to come up with an algorithm that looks at all these scenarios and selects one. The number of possibilities is large. Some chain of moves are inconsequential (0 score), while others are great (>0 score) or horrible (<0 score). Some chains result in a good/bad outcome sooner than others, and it makes sense the weigh them accordingly. Here is a proposal of how to weigh the possbilities and create score:

* At the end of the chain of moves, consider the worst and best scenarios.
* Otherwise, sum score over all possbilities, weighing each by the depth

Here is a recursive implementation of this algorithm:

In [ ]:
def tree_search_0(t,depth=1):
    if isinstance(t[0],list):
        return sum([tree_search(item,depth+1)/depth for item in t])
    else:
        return sum(t)
    
def tree_search(t,depth=1):
    if isinstance(t[0],list):
        return sum([tree_search(item,depth+1)/depth for item in t])
    else:
        return max(t)+min(t)

Quick test:

In [ ]:
list(map(tree_search,scores))

Finally a function to put it all together and select a move:

In [ ]:
def pick_move(board,player,depth=5,func=max):
    moves,scores=generate_moves(board,player,player,depth)
    result=list(map(tree_search,scores))
    move_index=result.index(func(result))
    return moves[move_index][0]

Lets test:

In [ ]:
pick_move(board,player_1)

Finally put it all together:

In [ ]:
def checkers_game_AI(depth=5):
    
    print ("Welcome to Checkers.")
    print ("--------------------")

    # Make a game board
    board_0=make_game_board()
    
    # Start with player 1
    player=player_1
    
    this_game_won=False
    while not this_game_won:
        # Draw the board
        draw_board(board_0)
        
        # Make a move
        if player==player_1:
            print("Player",player,"move:")
            take_move(board_0,player)
        else:
            the_move=pick_move(board_0,player_2,depth=depth)
            #print(the_move)
            move_piece(board_0,*the_move)
            
        # Check if the game has been won
        this_game_won=game_won(board_0)

       # Switch players
        player=switch_player(player)          

    print("Player 1 Pieces:", count_pieces(board_0,player_1))
    print("Player 2 Pieces:", count_pieces(board_0,player_2))
    
    if this_game_won:
        print("Winner is player:",this_game_won)

In [ ]:
checkers_game_AI()
